# Exploratory Stuff

In [46]:
from pprint import pprint
from pathlib import Path
import os

#pprint(dict(os.environ))

project = {"cwd": Path.cwd()}

project["xschem"]              = Path(os.environ["PDKPATH"]) / "libs.tech" / "xschem"
project["xschem_library"]      = Path("/foss/tools/xschem/share/xschem/xschem_library")

# Schematics and Symbols
project["inverter_schem"]      = project["cwd"] / "inverter.sch"
project["nmos_schem"]          = project["cwd"] / "nmos_charac_IPD413HW1.sch"
project["inverter_sym"]        = project["cwd"] / "inverter.sym"
project["inverter_tb_schem"]   = project["cwd"] / "inverter_tb.sch"

#!ls {project["xschem_library"]}
for name, path in project.items():
    if (not path.exists()):
        print(name, " no existe")
    #print(path.resolve(), path.exists())

In [47]:
class Xschem():
    def __init__(self, libraries: list):
        !echo set XSCHEM_LIBRARY_PATH {':'.join(map(str, libraries))} > {project["cwd"]}/xschemrc

    def edit(self, filename):
        !xschem {filename}

    def compile(self, filename):
        !xschem -n -x -o "spices" -q {filename} << "EOF\nquit\nEOF"

    def simulate(self, sim_file):
        pass

xschem_libraries= [
    project['xschem'],
    project['xschem_library'],
    project['cwd']
]

xschem = Xschem(xschem_libraries)

xschem.edit(project["nmos_schem"])
xschem.edit(project["inverter_schem"])

xschem.compile(project["nmos_schem"])
xschem.compile(project["inverter_schem"])

xschem [/foss/designs/jupyter] 
xschem [/foss/designs/jupyter] 




In [6]:
!ls spices

inverter.spice	nmos_charac_IPD413HW1.spice


In [7]:
!head spices/nmos_charac_IPD413HW1.spice

** sch_path: /foss/designs/jupyter/nmos_charac_IPD413HW1.sch
**.subckt nmos_charac_IPD413HW1 g1 d1
*.ipin g1
*.iopin d1
XM1 d1 g1 GND GND sky130_fd_pr__nfet_01v8 L=0.3 W=10 nf=1 ad='int((nf+1)/2) * W/nf * 0.29' as='int((nf+2)/2) * W/nf * 0.29'
+ pd='2*int((nf+1)/2) * (W/nf + 0.29)' ps='2*int((nf+2)/2) * (W/nf + 0.29)' nrd='0.29 / W' nrs='0.29 / W'
+ sa=0 sb=0 sd=0 mult=1 m=1
**** begin user architecture code




## PySpice usage

In [ ]:
from PySpice.Logging.Logging import setup_logging
logger = setup_logging()
from PySpice.Spice.Netlist import Circuit, SubCircuitFactory
from PySpice.Unit import *

test = Circuit("test")

with open("spices/nmos_charac_IPD413HW1.spice", "r") as f:
    raw_spice = f.read()
    print("Raw spice is: ", raw_spice)
    test.raw_spice = raw_spice
    
    print("Circuit is: ", test)


In [ ]:
from PySpice.Spice.Netlist import Circuit, SubCircuitFactory
from PySpice.Unit import *


class SubCircuit1(SubCircuitFactory):
    NAME = 'sub_circuit1'
    NODES = ('n1', 'n2')
    def __init__(self):
        super().__init__()
        self.R(1, 'n1', 'n2', 1@u_Ω)
        self.R(2, 'n1', 'n2', 2@u_Ω)

In [ ]:
circuit = Circuit("Test")

with open(project["inverter_schem"],"r") as f:
    print(f.read())

In [8]:
#test.simulator()
# We have to connect ngspice with this.

## Ngspyce

In [12]:
# import ngspyce
# OSError: /foss/tools/ngspice/ngspice/lib/libngspice.so.0: undefined symbol: cvprod

In [13]:
!ls /foss/tools/ngspice/ngspice/lib/

libngspice.la  libngspice.so.0	    ngspice
libngspice.so  libngspice.so.0.0.7  pkgconfig


## Skidl
Estoy tratando de simplificar la simulación, ojalá esto funcione



In [17]:
# This may require restart the kernel
!pip install skidl

# https://github.com/devbisme/skidl/blob/master/examples/spice-sim-intro/spice-sim-intro.ipynb

Defaulting to user installation because normal site-packages is not writeable


In [18]:
import skidl

## Klayout and Gds Viewing

In [19]:
#!klayout
# This works, but I don't have any gds :(

## Spice Reducer
This works when a large model tends to hang jupyter.

I think that this won't reduce common spice files :(

In [42]:
!cat $DESIGNS/../tools/iic-osic/iic-spice-model-red.py

#!/usr/bin/env python3
# ========================================================================
# SPDX-FileCopyrightText: 2021-2022 Harald Pretl
# Johannes Kepler University, Institute for Integrated Circuits
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# SPDX-License-Identifier: Apache-2.0
#
# This script traverses SPICE model files (e.g. from SKY130) and
# extracts only the wanted model section, removes all comments and
# empty lines, and resolves all includes so that a flat model file
# re

In [43]:
!iic-spice-model-red.py ./spices/inverter.spice


Model file ./spices/inverter.spice.tt.red written.


In [44]:
!ls -lh spices/

total 8.0K
-rw-r--r-- 1 designer designers  662 May 17 05:20 inverter.spice
-rw-r--r-- 1 designer designers    0 May 17 05:36 inverter.spice.tt.red
-rw-r--r-- 1 designer designers 1.2K May 17 05:20 nmos_charac_IPD413HW1.spice
